Katie Briggs
DSC550-302
Density Based Clustering: DENCLUE


# 2. Density Based Clustering: DENCLUE

Write a script to implement the DENCLUE density-based clustering algorithm Algorithm 15.2 in chapter 15.
The script should take as input a dataset D, the minimum density ξ, the tolerance for convergence ϵ, and the width h.
Do not make any assumptions about the data (i.e., column names, etc), 
except that the last column gives the "true" cluster id.

Run your script on the iris.txt dataset, with ϵ=0.0001. Your script should output the following:

The number of clusters, and the size of each cluster.
The density attractor, followed by the set of point in that cluster.
Purity of the clustering, based on the true id.


In [2]:
# import libraries needed

import numpy as np
from sklearn.base import BaseEstimator, ClusterMixin
import networkx as nx
from sklearn.datasets import load_iris
import pandas as pd


In [32]:
# read in iris dataframe from notebook

iris = load_iris()
iris1 = pd.DataFrame(iris.data, columns=iris['feature_names'])
data = iris1[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)']]

In [33]:
# reference https://github.com/mgarrett57/DENCLUE/blob/master/denclue.py as example
# create function
def climb(x_t, X, W=None, h=0.1, eps=1e-7):
    error = 99.
    prob = 0.
    x_l1 = np.copy(x_t) 
    radius_new = 0.
    radius_old = 0.
    radius_twiceold = 0.
    iters = 0.
    while True:
        radius_thriceold = radius_twiceold
        radius_twiceold = radius_old
        radius_old = radius_new
        x_l0 = np.copy(x_l1)       #X(t)
        x_l1, density = _step(x_l0, X, W=W, h=h)
        error = density - prob
        prob = density
        radius_new = np.linalg.norm(x_l1 - x_l0)
        radius = radius_thriceold + radius_twiceold + radius_old + radius_new
        iters += 1
        if iters > 3 and error < eps:
            break
    return [x_l1, prob, radius]

In [59]:
def den_attr(x_l0, X, W=None, h=0.1):
    n = X.shape[0]
    d = X.shape[1]
    superweight = 0.  # superweight is the kernel X weight for each item
    x_l1 = np.zeros((1, d))
    if W is None:
        W = np.ones((n, 1))
    else:
        W = W
    for j in range(n):
        kernel = kernelize(x_l0, X[j], h, d)
        kernel = kernel * W[j] / (h ** d)
        superweight = superweight + kernel
        x_l1 = x_l1 + (kernel * X[j])
    x_l1 = x_l1 / superweight
    density = superweight / np.sum(W)
    return [x_l1, density]

# create function to Gaussian kernel

def kernelize(x, y, h, degree):
    kernel = np.exp(-(np.linalg.norm(x - y) / h) ** 2. / 2.) / ((2. * np.pi) ** (degree / 2))
    return kernel


# create function to start DENCLUE algorithm

def DENCLUE(D, h, xi, eps):
    rows = D.shape[0]
    A = np.zeros((rows,D.shape[1]))
    labels = -np.ones(rows)
    dens = np.zeros((rows,1))
    degree = D.shape[1]
    for i in range(rows):
        x_mark = findattractor(D[i], D, h, eps)
        dens[i] = density(x_mark, D, h, degree)
        if dens[i]>=xi:
            A[i]= x_mark
    print(A, dens)
    
    
def __init__(self, h = None, eps = 0.0001, min_density = 0, metric = 'euclidean'):
    self.h = h
    self.eps = eps
    self.min_density = min_density
    self.metric = metric
    



In [60]:

def classify(self, X, y=None, sample_weight=None):
    if not self.eps>0.0:
        raise ValueError("eps must be positive")
    self.n_samples = X.shape[0]
    self.n_features = X.shape[1]
    density_attractors = np.zeros((self.n_samples, self.n_features))
    radii = np.zeros((self.n_samples, 1))
    density = np.zeros((self.n_samples, 1))
    if self.h is None:
        self.h = np.std(X) / 5
        if sample_weight is None:
           sample_weight = np.ones((self.n_samples, 1))
        else:
            sample_weight = sample_weight
            labels = -np.ones(X.shape[0])
        for i in range(self.n_samples):
             density_attractors[i], density[i], radii[i] = _hill_climb(X[i], X, W=sample_weight, h = self.h, eps= self.eps)
             cluster_info = {}
             num_clusters = 0
             cluster_info[num_clusters] = {'instances': [0], 'centroid': np.atleast_2d(density_attractors[0])}
             g_clusters = nx.Graph()
        for j1 in range(self.n_samples):
            g_clusters.add_node(j1, attr_dict = {'attractor': density_attractors[j1], 
                                         'radius': radii[j1], 'density': density[j1]})
        for j1 in range(self.n_samples):
            for j2 in (x for x in range(self.n_samples) if x != j1):
                if g_clusters.has_edge(j1, j2):
                    continue
                diff = np.linalg.norm(g_clusters.node[j1]['attractor'] - g_clusters.node[j2]['attractor'])
                if diff <= (g_clusters.node[j1]['radius'] + g_clusters.node[j1]['radius']):
                    g_clusters.add_edge(j1, j2)
    clusters = list(nx.connected_component_subgraphs(g_clusters))
    num_clusters = 0
    for clust in clusters:
        max_instance = max(clust, key=lambda x: clust.node[x]['density'])
        max_density = clust.node[max_instance]['density']
        max_centroid = clust.node[max_instance]['attractor']
        complete = False
        c_size = len(clust.nodes())
        if clust.number_of_edges() == (c_size * (c_size - 1)) / 2.:
            complete = True
        cluster_info[num_clusters] = {'instances': clust.nodes(),'size': c_size, 
                                      'centroid': max_centroid, 'density': max_density, 'complete': complete}
        if max_density >= self.min_density:
            labels[clust.nodes()] = num_clusters
        num_clusters += 1
    self.clust_info_ = cluster_info
    self.labels_ = labels
    return self

data = load_iris()
data1 = np.array(data)
data1.shape
print(data)
samples = np.mat(data1, dtype=None)  # example  A[B[:,0]==0, 0]
true_label = data1[: , 0]
labels = list(set(true_labels))
true_ID = np.zeros((3, 50))
index = range(len(true_labels))
for i in range(len(labels)):
    true_id[i]=[j for j in index if true_labels[j]==labels[i]]
d = DENCLUE(0.25, 0.0001)
d.classify(samples)
right_num = 0
for i in range(len(d.clust_info_)):
    bestlens=0
    clust_set = set(d.clust_info_[i]['instances'])
    for j in range(len(labels)):
        true_set=set(true_ID[j])
        and_set= clust_set&true_set
        if len(list(and_set))>bestlens:
            bestlens=len(list(and_set))
    right_num+=bestlens
    
print(d.clust_info_, float(right_num)/len(samples))

In [90]:
data = load_iris()
data1 = np.array(data)
data1.shape
print(data1)
samples = np.mat(data1, dtype=None)
true_label = data[: , -1]
labels = list(set(true_labels))
true_ID = np.zeros((3, 50))
index = range(len(true_labels))
for i in range(len(labels)):
    true_id[i]=[j for j in index if true_labels[j]==labels[i]]
d = DENCLUE(0.25, 0.0001)
d.classify(samples)
right_num = 0
for i in range(len(d.clust_info_)):
    bestlens=0
    clust_set = set(d.clust_info_[i]['instances'])
    for j in range(len(labels)):
        true_set=set(true_ID[j])
        and_set= clust_set&true_set
        if len(list(and_set))>bestlens:
            bestlens=len(list(and_set))
    right_num+=bestlens
    
print(d.clust_info_, float(right_num)/len(samples))

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

TypeError: unhashable type: 'slice'